In [27]:
# Kaggle: Do NOT install torch/torchvision here!
!pip install pyradiomics SimpleITK nibabel scikit-learn xgboost lightgbm timm einops

import os
import gc
import pickle
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from einops import rearrange
import SimpleITK as sitk
import nibabel as nib
from PIL import Image

from radiomics import featureextractor
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')

##############################################################
# MODALITY-SPECIFIC NORMALIZATION
##############################################################

def normalize_image(volume: np.ndarray, modality: str) -> np.ndarray:
    modality = modality.upper().strip()
    if "CTA" in modality or "CT" in modality:
        center, width = 250, 600
        min_val = center - width // 2
        max_val = center + width // 2
        if volume.max() <= 1.0:
            volume = volume * 500 - 100
        volume = np.clip(volume, min_val, max_val)
        volume = (volume - min_val) / float(max_val - min_val)
        volume = np.clip(volume, 0, 1)
    elif "MRA" in modality or "MRI" in modality:
        mean_val = np.mean(volume)
        std_val = np.std(volume)
        if std_val < 1e-6:
            std_val = 1.0
        volume = (volume - mean_val) / std_val
    else:
        min_val = np.min(volume)
        max_val = np.max(volume)
        if max_val - min_val < 1e-6:
            volume = np.zeros_like(volume)
        else:
            volume = (volume - min_val) / (max_val - min_val)
    return volume.astype(np.float32)

##############################################################
# CONFIGURATION
##############################################################

class Config:
    PNG_ROOT = "/kaggle/input/rsna-2025-intracranial-aneurysm-png-224x224/cvt_png"
    SEGMENTATION_ROOT = "/kaggle/input/rsna-intracranial-aneurysm-detection/segmentations"
    TRAIN_CSV = "/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv"
    BATCH_SIZE = 4
    IMG_SIZE = 224
    SEED = 42
    N_FOLDS = 5
    RADIOMICS_FEATURES = True
    DEEP_FEATURES = True
    VIT_FEATURES = True
    ARTERIES = [
        'Left Infraclinoid Internal Carotid Artery',
        'Right Infraclinoid Internal Carotid Artery', 
        'Left Supraclinoid Internal Carotid Artery',
        'Right Supraclinoid Internal Carotid Artery',
        'Left Anterior Cerebral Artery',
        'Right Anterior Cerebral Artery',
        'Anterior Communicating Artery',
        'Left Middle Cerebral Artery',
        'Right Middle Cerebral Artery',
        'Left Posterior Communicating Artery',
        'Right Posterior Communicating Artery',
        'Basilar Tip',
        'Other Posterior Circulation'
    ]

##############################################################
# DATASET
##############################################################

class AneurysmDataset(Dataset):
    def __init__(self, series_data: pd.DataFrame, config: Config, mode: str = 'train', transform=None):
        self.series_data = series_data
        self.config = config
        self.mode = mode
        self.transform = transform
    def __len__(self):
        return len(self.series_data)
    def __getitem__(self, idx):
        row = self.series_data.iloc[idx]
        series_id = row['SeriesInstanceUID']
        modality = row.get('Modality', 'MRA')
        png_series_path = self._find_png_series(series_id)
        if png_series_path is None:
            return {
                'volume': np.zeros((1, self.config.IMG_SIZE, self.config.IMG_SIZE), dtype=np.float32),
                'mask': None,
                'series_id': series_id,
                'modality': modality,
                'labels': np.zeros(14, dtype=np.float32) if self.mode == 'train' else None
            }
        volume = self._load_png_series(png_series_path, modality)
        mask = self._load_segmentation_mask(series_id)
        if self.transform:
            volume = self.transform(volume)
        return {
            'volume': volume,
            'mask': mask,
            'series_id': series_id,
            'modality': modality,
            'labels': self._get_labels(row) if self.mode == 'train' else None
        }
    def _find_png_series(self, series_id: str) -> Optional[Path]:
        for artery_folder in Path(self.config.PNG_ROOT).iterdir():
            if artery_folder.is_dir():
                series_folder = artery_folder / series_id
                if series_folder.exists():
                    return series_folder
        return None
    def _load_png_series(self, series_path: Path, modality: str) -> np.ndarray:
        png_files = sorted([f for f in series_path.glob('*.png')])
        if not png_files:
            return np.zeros((1, self.config.IMG_SIZE, self.config.IMG_SIZE))
        slices = []
        for png_file in png_files:
            img = Image.open(png_file).convert('L')
            img_array = np.array(img, dtype=np.float32)
            if img_array.max() > 1.0:
                img_array = img_array / 255.0
            slices.append(img_array)
        volume = np.stack(slices, axis=0)
        volume = normalize_image(volume, modality)
        return volume
    def _load_segmentation_mask(self, series_id: str) -> Optional[np.ndarray]:
        mask_path = Path(self.config.SEGMENTATION_ROOT) / f"{series_id}.nii"
        if mask_path.exists():
            try:
                mask_nii = nib.load(str(mask_path))
                mask = mask_nii.get_fdata().astype(np.uint8)
                return mask
            except Exception as e:
                print(f"Error loading mask for {series_id}: {e}")
        return None
    def _get_labels(self, row: pd.Series) -> np.ndarray:
        labels = []
        labels.append(1 if row.get('Aneurysm Present', 0) == 1 else 0)
        for artery in self.config.ARTERIES:
            artery_short = artery.replace(' ', '').replace('Internal', 'Infra')[:15] if len(artery) > 15 else artery
            label_val = 0
            for col in row.index:
                if artery in col or artery_short in col:
                    label_val = 1 if row[col] == 1 else 0
                    break
            labels.append(label_val)
        return np.array(labels, dtype=np.float32)

##############################################################
# 3D UNet, MedicalViT, RadiomicsExtractor, FeaturePipeline, and EnsembleModel (as in your previous code)
# Copy the same implementations as above, or ask if you want them pasted here in full as well (they are long).
##############################################################

# Main pipeline and scoring functions follow as before.




# ======================================================================
# 3D U-NET MODEL FOR DEEP FEATURE EXTRACTION
# ======================================================================
class DoubleConv(nn.Module):
    """Double convolution block for U-Net"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet3D(nn.Module):
    """3D U-Net for feature extraction and segmentation"""

    def __init__(self, n_channels=1, n_classes=14, feature_dim=512):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        # Encoder
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool3d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool3d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool3d(2), DoubleConv(256, 512))

        # Feature extraction
        self.feature_pool = nn.AdaptiveAvgPool3d(1)
        self.feature_fc = nn.Linear(512, feature_dim)

        # Classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(feature_dim, n_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Add channel dimension if needed
        if len(x.shape) == 4:
            x = x.unsqueeze(1)  # [B, 1, D, H, W]

        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)

        # Feature extraction
        features = self.feature_pool(x4).squeeze(-1).squeeze(-1).squeeze(-1)
        features = self.feature_fc(features)

        # Classification
        outputs = self.classifier(features)

        return outputs, features

# ======================================================================
# VISION TRANSFORMER FOR MEDICAL IMAGING
# ======================================================================
class MedicalViT(nn.Module):
    """Vision Transformer adapted for 3D medical volumes"""

    def __init__(self, 
                 image_size=224,
                 patch_size=16,
                 num_slices=32,
                 num_classes=14,
                 dim=768,
                 depth=12,
                 heads=12,
                 mlp_dim=3072,
                 dropout=0.1):
        super().__init__()

        self.image_size = image_size
        self.patch_size = patch_size
        self.num_slices = num_slices
        self.num_classes = num_classes

        # Load pretrained 2D ViT and adapt for 3D
        self.vit_2d = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)

        # 3D to 2D adapter - process each slice
        self.slice_processor = nn.Identity()

        # Feature aggregation across slices
        self.slice_attention = nn.MultiheadAttention(dim, heads, dropout=dropout)
        self.slice_norm = nn.LayerNorm(dim)

        # Final classifier
        self.classifier = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Dropout(dropout),
            nn.Linear(dim, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: [B, D, H, W] or [B, 1, D, H, W]
        if len(x.shape) == 5:
            x = x.squeeze(1)  # Remove channel dim if present

        batch_size, depth, height, width = x.shape

        # Process each slice through 2D ViT
        slice_features = []

        # Limit number of slices to prevent memory issues
        max_slices = min(depth, self.num_slices)
        slice_indices = np.linspace(0, depth-1, max_slices).astype(int)

        for i in slice_indices:
            # Get slice and add channel dimension
            slice_img = x[:, i:i+1, :, :]  # [B, 1, H, W]

            # Convert to 3-channel for pretrained model
            slice_img = slice_img.repeat(1, 3, 1, 1)  # [B, 3, H, W]

            # Extract features using pretrained ViT
            slice_feat = self.vit_2d(slice_img)  # [B, 768]
            slice_features.append(slice_feat)

        # Stack slice features
        slice_features = torch.stack(slice_features, dim=1)  # [B, num_slices, 768]

        # Apply attention across slices
        slice_features = slice_features.transpose(0, 1)  # [num_slices, B, 768]
        attended_features, _ = self.slice_attention(slice_features, slice_features, slice_features)
        attended_features = self.slice_norm(attended_features + slice_features)

        # Global average pooling across slices
        global_features = attended_features.mean(dim=0)  # [B, 768]

        # Classification
        outputs = self.classifier(global_features)

        return outputs, global_features

# ======================================================================
# ENHANCED RADIOMICS EXTRACTOR
# ======================================================================
class RadiomicsExtractor:
    """Extract radiomics features using PyRadiomics with modality awareness"""

    def __init__(self):
        # Initialize radiomics extractor
        self.extractor = featureextractor.RadiomicsFeatureExtractor()

        # Configure settings
        settings = {
            'binWidth': 25,
            'resampledPixelSpacing': None,
            'interpolator': 'sitkLinear',
            'verbose': False
        }

        self.extractor.enableAllImageTypes()
        self.extractor.enableAllFeatures()

        for key, value in settings.items():
            self.extractor.settings[key] = value

    def extract_features(self, volume: np.ndarray, mask: np.ndarray = None, modality: str = "MRA") -> Dict:
        """Extract radiomics features from volume with modality-specific settings"""

        # Convert to SimpleITK images
        volume_sitk = sitk.GetImageFromArray(volume.astype(np.float32))

        if mask is not None:
            mask_sitk = sitk.GetImageFromArray(mask.astype(np.uint8))
        else:
            # Create full volume mask
            mask_sitk = sitk.GetImageFromArray(np.ones_like(volume).astype(np.uint8))

        # Set spacing (assume isotropic 1mm)
        volume_sitk.SetSpacing((1.0, 1.0, 1.0))
        mask_sitk.SetSpacing((1.0, 1.0, 1.0))

        try:
            # Extract features
            features = self.extractor.execute(volume_sitk, mask_sitk)

            # Filter out non-feature keys and add modality prefix
            radiomics_features = {}
            for key, value in features.items():
                if key.startswith(('original_', 'wavelet-', 'log-', 'square-', 'squareroot-', 'logarithm-', 'exponential-')):
                    try:
                        radiomics_features[f"{modality}_{key}"] = float(value)
                    except (ValueError, TypeError):
                        continue

            return radiomics_features

        except Exception as e:
            print(f"Radiomics extraction failed for {modality}: {e}")
            return {}

# ======================================================================
# ENHANCED FEATURE PIPELINE
# ======================================================================
class FeaturePipeline:
    """Complete feature extraction pipeline with ViT integration"""

    def __init__(self, config: Config):
        self.config = config
        self.deep_model = None
        self.vit_model = None
        self.radiomics_extractor = RadiomicsExtractor()
        self.scaler = StandardScaler()
        self.feature_selector = SelectKBest(f_classif, k=100)  # Increased for more features

    def load_models(self, deep_model_path: str = None, vit_model_path: str = None):
        """Load or initialize deep learning models"""

        # Load 3D U-Net
        if self.config.DEEP_FEATURES:
            self.deep_model = UNet3D()
            if deep_model_path and os.path.exists(deep_model_path):
                self.deep_model.load_state_dict(torch.load(deep_model_path))
            self.deep_model.eval()

        # Load ViT
        if self.config.VIT_FEATURES:
            self.vit_model = MedicalViT()
            if vit_model_path and os.path.exists(vit_model_path):
                self.vit_model.load_state_dict(torch.load(vit_model_path))
            self.vit_model.eval()

    def extract_features(self, dataset: AneurysmDataset) -> Tuple[np.ndarray, np.ndarray]:
        """Extract all features from dataset"""

        all_features = []
        all_labels = []

        dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

        for i, batch in enumerate(dataloader):
            if batch is None:
                continue

            volume = batch['volume'][0].numpy()  # Remove batch dimension
            mask = batch['mask'][0] if batch['mask'][0] is not None else None
            modality = batch['modality'][0]
            labels = batch['labels'][0] if batch['labels'][0] is not None else None

            print(f"Processing sample {i+1}/{len(dataloader)}: {batch['series_id'][0]} ({modality})")

            # Extract features
            features = {}

            # Deep features (3D U-Net)
            if self.config.DEEP_FEATURES and self.deep_model:
                try:
                    with torch.no_grad():
                        volume_tensor = torch.FloatTensor(volume).unsqueeze(0)
                        _, deep_feats = self.deep_model(volume_tensor)
                        deep_feats = deep_feats.squeeze().numpy()

                        for j, feat in enumerate(deep_feats):
                            features[f'{modality}_deep_feature_{j}'] = feat
                except Exception as e:
                    print(f"Deep feature extraction failed: {e}")

            # ViT features
            if self.config.VIT_FEATURES and self.vit_model:
                try:
                    with torch.no_grad():
                        volume_tensor = torch.FloatTensor(volume).unsqueeze(0)
                        _, vit_feats = self.vit_model(volume_tensor)
                        vit_feats = vit_feats.squeeze().numpy()

                        for j, feat in enumerate(vit_feats):
                            features[f'{modality}_vit_feature_{j}'] = feat
                except Exception as e:
                    print(f"ViT feature extraction failed: {e}")

            # Radiomics features
            if self.config.RADIOMICS_FEATURES:
                try:
                    mask_array = mask.numpy() if mask is not None else None
                    radiomics_feats = self.radiomics_extractor.extract_features(volume, mask_array, modality)
                    features.update(radiomics_feats)
                except Exception as e:
                    print(f"Radiomics extraction failed: {e}")

            if features:
                all_features.append(features)
                if labels is not None:
                    all_labels.append(labels.numpy())

            # Memory cleanup
            if i % 10 == 0:
                gc.collect()

        # Convert to arrays
        if all_features:
            print(f"Creating feature matrix from {len(all_features)} samples...")
            feature_df = pd.DataFrame(all_features).fillna(0)
            X = feature_df.values
            y = np.array(all_labels) if all_labels else None

            print(f"Feature matrix shape: {X.shape}")
            return X, y

        return np.array([]), np.array([])

# ======================================================================
# ENHANCED ENSEMBLE MODEL
# ======================================================================
class EnsembleModel:
    """Stacking ensemble with multiple base models optimized for medical imaging"""

    def __init__(self, config: Config):
        self.config = config

        # Base models with medical imaging optimizations
        self.base_models = {
            'rf': RandomForestClassifier(
                n_estimators=200,
                max_depth=10, 
                min_samples_split=5,
                random_state=config.SEED,
                n_jobs=-1
            ),
            'xgb': xgb.XGBClassifier(
                n_estimators=200,
                max_depth=6,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=config.SEED,
                eval_metric='logloss'
            ),
            'lgb': lgb.LGBMClassifier(
                n_estimators=200,
                max_depth=6,
                learning_rate=0.1,
                feature_fraction=0.8,
                bagging_fraction=0.8,
                random_state=config.SEED,
                verbose=-1
            )
        }

        # Meta-learner
        self.meta_model = LogisticRegression(
            random_state=config.SEED,
            max_iter=1000,
            C=0.1
        )

        self.is_fitted = False

    def fit(self, X: np.ndarray, y: np.ndarray):
        """Fit ensemble model with improved stacking"""

        print("Training ensemble model with stacking...")

        # Generate meta-features using cross-validation
        n_models = len(self.base_models)
        n_labels = y.shape[1]
        meta_features = np.zeros((X.shape[0], n_models * n_labels))

        kf = StratifiedKFold(n_splits=self.config.N_FOLDS, shuffle=True, random_state=self.config.SEED)

        for fold, (train_idx, val_idx) in enumerate(kf.split(X, y[:, 0])):  # Stratify on global label
            print(f"Processing fold {fold + 1}/{self.config.N_FOLDS}")

            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            for i, (name, model) in enumerate(self.base_models.items()):
                print(f"  Training {name}...")

                # Train base model
                if hasattr(model, 'fit'):
                    model.fit(X_train, y_train)

                # Predict on validation set
                if hasattr(model, 'predict_proba'):
                    val_pred = model.predict_proba(X_val)
                else:
                    val_pred = model.predict(X_val)

                # Store meta-features
                start_idx = i * n_labels
                end_idx = (i + 1) * n_labels

                if isinstance(val_pred, list) and len(val_pred) == n_labels:
                    # Multi-output case (sklearn style)
                    for j in range(n_labels):
                        if hasattr(val_pred[j], 'shape') and len(val_pred[j].shape) > 1:
                            meta_features[val_idx, start_idx + j] = val_pred[j][:, 1]
                        else:
                            meta_features[val_idx, start_idx + j] = val_pred[j]
                else:
                    # Single output case
                    if len(val_pred.shape) == 2 and val_pred.shape[1] == n_labels:
                        meta_features[val_idx, start_idx:end_idx] = val_pred
                    else:
                        # Fallback: use predictions as is
                        meta_features[val_idx, start_idx:start_idx+1] = val_pred.reshape(-1, 1)

        # Train final base models on full dataset
        print("Training final base models...")
        for name, model in self.base_models.items():
            print(f"  Final training {name}...")
            model.fit(X, y)

        # Train meta-learner
        print("Training meta-learner...")
        self.meta_model.fit(meta_features, y)

        self.is_fitted = True
        print("Ensemble training completed!")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Make predictions using ensemble"""

        if not self.is_fitted:
            raise ValueError("Model must be fitted before making predictions")

        # Generate meta-features
        meta_features = []

        for name, model in self.base_models.items():
            if hasattr(model, 'predict_proba'):
                pred = model.predict_proba(X)
            else:
                pred = model.predict(X)

            if isinstance(pred, list):
                # Multi-output case
                meta_feat = np.column_stack([
                    p[:, 1] if len(p.shape) > 1 and p.shape[1] > 1 else p.ravel()
                    for p in pred
                ])
            else:
                # Single output case
                if len(pred.shape) == 1:
                    meta_feat = pred.reshape(-1, 1)
                else:
                    meta_feat = pred

            meta_features.append(meta_feat)

        meta_X = np.column_stack(meta_features)

        # Final prediction
        if hasattr(self.meta_model, 'predict_proba'):
            final_pred = self.meta_model.predict_proba(meta_X)
            if isinstance(final_pred, list):
                return np.column_stack([p[:, 1] for p in final_pred])
            else:
                return final_pred
        else:
            return self.meta_model.predict(meta_X)

# ======================================================================
# EVALUATION FUNCTIONS
# ======================================================================
def calculate_competition_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Calculate weighted competition score"""

    try:
        # Global aneurysm AUC (weight = 13)
        if len(np.unique(y_true[:, 0])) > 1:
            global_auc = roc_auc_score(y_true[:, 0], y_pred[:, 0])
        else:
            global_auc = 0.5

        # Per-artery AUCs (weight = 1 each)
        artery_aucs = []
        for i in range(1, min(y_true.shape[1], y_pred.shape[1])):
            if len(np.unique(y_true[:, i])) > 1:  # Check if both classes present
                auc = roc_auc_score(y_true[:, i], y_pred[:, i])
                artery_aucs.append(auc)

        # Weighted score
        if len(artery_aucs) > 0:
            total_weight = 13 + len(artery_aucs)
            weighted_score = (13 * global_auc + sum(artery_aucs)) / total_weight
        else:
            weighted_score = global_auc

        return weighted_score

    except Exception as e:
        print(f"Error calculating competition score: {e}")
        return 0.0

# ======================================================================
# MAIN PIPELINE
# ======================================================================
def main():
    """Main execution pipeline with enhanced features"""

    print("🔬 Starting Enhanced Multimodal Aneurysm Detection Pipeline...")
    print("✨ Features: Modality-specific normalization, 3D U-Net, ViT, Advanced ensemble")

    config = Config()

    # Load training data
    print("📊 Loading training data...")
    train_df = pd.read_csv(config.TRAIN_CSV)
    print(f"Loaded {len(train_df)} training samples")

    # Display modality distribution
    if 'Modality' in train_df.columns:
        print("📈 Modality distribution:")
        print(train_df['Modality'].value_counts())

    # Create dataset
    train_dataset = AneurysmDataset(train_df, config, mode='train')
    print(f"Created dataset with {len(train_dataset)} samples")

    # Initialize feature pipeline
    print("🔧 Initializing enhanced feature extraction pipeline...")
    feature_pipeline = FeaturePipeline(config)
    feature_pipeline.load_models()

    # Extract features
    print("🎯 Extracting features (this may take a while)...")
    X, y = feature_pipeline.extract_features(train_dataset)

    if X.shape[0] == 0:
        print("❌ No features extracted. Check data paths and format.")
        return

    print(f"✅ Extracted features: {X.shape}")
    print(f"✅ Labels shape: {y.shape}")

    # Preprocess features
    print("🔄 Preprocessing features...")
    X_scaled = feature_pipeline.scaler.fit_transform(X)

    if X_scaled.shape[1] > 100:
        X_selected = feature_pipeline.feature_selector.fit_transform(X_scaled, y[:, 0])
    else:
        X_selected = X_scaled

    print(f"✅ Final feature shape: {X_selected.shape}")

    # Train ensemble model
    print("🤖 Training enhanced ensemble model...")
    ensemble = EnsembleModel(config)
    ensemble.fit(X_selected, y)

    # Cross-validation evaluation
    print("📈 Evaluating model with cross-validation...")
    cv_scores = []

    kf = StratifiedKFold(n_splits=config.N_FOLDS, shuffle=True, random_state=config.SEED)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_selected, y[:, 0])):
        print(f"Evaluating fold {fold + 1}/{config.N_FOLDS}...")

        X_train, X_val = X_selected[train_idx], X_selected[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Train fold model
        fold_ensemble = EnsembleModel(config)
        fold_ensemble.fit(X_train, y_train)

        # Predict
        y_pred = fold_ensemble.predict(X_val)

        # Calculate score
        score = calculate_competition_score(y_val, y_pred)
        cv_scores.append(score)

        print(f"  Fold {fold + 1} Score: {score:.4f}")

    print(f"🎊 Mean CV Score: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

    # Save model and pipeline
    print("💾 Saving enhanced model pipeline...")
    model_artifacts = {
        'ensemble': ensemble,
        'scaler': feature_pipeline.scaler,
        'selector': feature_pipeline.feature_selector,
        'config': config,
        'cv_scores': cv_scores,
        'feature_names': list(range(X_selected.shape[1]))
    }

    with open('/kaggle/working/enhanced_aneurysm_ensemble.pkl', 'wb') as f:
        pickle.dump(model_artifacts, f)

    print("✅ Enhanced pipeline completed successfully!")
    print(f"🏆 Final Performance: {np.mean(cv_scores):.4f} AUC-ROC")

if __name__ == "__main__":
    main()


🔬 Starting Enhanced Multimodal Aneurysm Detection Pipeline...
✨ Features: Modality-specific normalization, 3D U-Net, ViT, Advanced ensemble
📊 Loading training data...
Loaded 4348 training samples
📈 Modality distribution:
Modality
CTA           1808
MRA           1252
MRI T2         983
MRI T1post     305
Name: count, dtype: int64
Created dataset with 4348 samples
🔧 Initializing enhanced feature extraction pipeline...
🎯 Extracting features (this may take a while)...


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>